In [1]:
############ Reprendre avec les codes de autoscout24

In [2]:
##Commencement : importer les packages

import requests 
import pandas as pd
import json   
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import re


In [3]:
website = "https://www.bymycar.fr/v/voiture-occasion?itemsperpage=24&sortingorder=ascending&filter=marque-du-vehicule%3Amercedes"

In [4]:
response = requests.get(website)

In [5]:
soup = BeautifulSoup(response.content,"html.parser")

In [6]:
results = soup.find_all("a",{"class" : "product-item" })

In [7]:
len(results)

24

In [8]:
############### exemple 

In [9]:
############## création boucle pour récupérer :


# Modèle 

# détails du modèle

# Prix 

# type de véhicule ( consommation )

# Nombre de kilométrage 

# Année 

# boite de vitesse 

# (complémentaire : récuperer le co2 pour une méilleure analyse)


In [10]:
modele = []
detail = []
prix = []
type_v = []
boite_v = []

for i in range(1,10):
    
    # website dans une variable
    website = "https://www.bymycar.fr/v/voiture-occasion?filter=marque-du-vehicule%3Amercedes&sortingorder=ascending&itemsperpage=24&display=grid&pagenumber="+str(i)

    # resquest au website
    response = requests.get(website)
    
    # soup object
    soup = BeautifulSoup(response.content,"html.parser")
    
    #result
    results = soup.find_all("a",{"class" : "product-item"})
    

    for result in results :

      try:

        modele.append(result.find("h2",{"class":"product__title"}).get_text())

      except:
        modele.append('n/a')

      try:

        detail.append(result.find("p",{"class" : "product__type" }).get_text())

      except:
        detail.append('n/a')  

      try:

        prix.append(result.find("div",{"class":"product__price_current"}).get_text())

      except:
        prix.append('n/a')

      try:

        type_v.append(result.find("ul",{"class" : "product__characteristics" }).get_text())

      except:
        type_v.append('n/a')      

      try:

        boite_v.append(result.find("ul",{"class" : "product__features_list" }).get_text())

      except:
        boite_v.append('n/a')    


In [11]:
car_dealer = pd.DataFrame({"Modele":modele , "Info vehicule":detail ,"Prix" :prix,
                           "type_vehicule":type_v,"puissance":boite_v})

In [12]:
##### clean les colonnes du dataframe
car_dealer["type_vehicule"] = car_dealer["type_vehicule"].str.lstrip("\n")

In [13]:
car_dealer["Nombre de kilometre"] = car_dealer["type_vehicule"].str.split("km", expand = True)[0]

In [14]:
def extract_after_word(string, word):
    if word in string:
        return string[string.index(word)+len(word):string.index(word)+len(word)+16]
    else:
        return None

In [15]:
car_dealer['Annee_sortie'] = car_dealer['type_vehicule'].apply(lambda x: extract_after_word(x,"km"))

In [16]:
def keep_digits(string):
    return ''.join(filter(str.isdigit, string))

In [17]:
car_dealer['Annee_sortie'] = car_dealer['Annee_sortie'].apply(keep_digits)

In [18]:
del car_dealer["type_vehicule"]

In [19]:
def extract_before_word(string, word):
    if word in string:
        return string[string.index(word)-4:string.index(word)]
    else:
        return None

In [20]:
car_dealer['puissance_chevaux'] = car_dealer['puissance'].apply(lambda x: extract_before_word(x, "cv"))

In [21]:
del car_dealer["puissance"]

In [22]:
car_dealer['Marque'] = "Mercedes"

In [23]:
car_dealer = car_dealer.reindex(columns=['Marque', 'Modele', 'Info vehicule','Prix', 'Nombre de kilometre', 'Annee_sortie',
                                         'puissance_chevaux'])


In [24]:
car_dealer['Prix'] = car_dealer['Prix'].apply(keep_digits)

In [25]:
car_dealer

,Marque,Modele,Info vehicule,Prix,Nombre de kilometre,Annee_sortie,puissance_chevaux
0,Mercedes,Classe B 200 d 8G-DCT,DIESEL,33990,62568,2000,150
1,Mercedes,GLC 300 de 9G-Tronic 4Matic,Hybride,72900,10000,2022,194
2,Mercedes,EQC 400 4Matic,Électrique,69998,28996,2020,408
3,Mercedes,Classe V Long 220 d 7G-TRONIC PLUS,DIESEL,36793,169744,2016,163
4,Mercedes,Classe CLS 400d 4Matic 9G-Tronic,DIESEL,89900,23500,2021,330
...,...,...,...,...,...,...,...
211,Mercedes,Classe A 180 d 7G-DCT,DIESEL,32990,20710,2021,116
212,Mercedes,Classe A 180 d 8G-DCT,DIESEL,46900,10000,2022,116
213,Mercedes,Classe GLE Coupé 350 de 9G-Tronic 4Matic,Hybride,92199,35588,2021,194
214,Mercedes,GLC Coupé 300 e 9G-Tronic 4Matic,Hybride,64861,64586,2020,211
